<a href="https://colab.research.google.com/github/Juniorexz/Codigo/blob/master/Machine_Learning_Validando_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

In [3]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [5]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 58.00%


A acurácia do dummy stratified foi de 50.96%

In [6]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [7]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)


Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 71.92%


In [8]:
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 3 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 3 = [74.99, 76.57]


A acurácia sobe para 76.84%, quase 77%! Então, aquela decisão de julgar isso como bom ou ruim em comparação ao que obtínhamos com Dummy Classifier está melhor, entretanto, ainda estamos vulneráveis a tomar uma decisão baseada em uma aleatoriedade. Não faz sentido validar, julgar se é bom ou ruim, de acordo com um número aleatório. Queremos minimizar o efeito dessa aleatoriedade, dessa escolha aleatória do treino, do teste e da DecisionTreeClassifier, na nossa decisão.

No entanto, se rodamos o treino e o teste uma única vez, corremos o risco de os ter separado mal. Precisamos de outra forma de treinar e testar mais de uma vez para que não tenhamos uma estimativa única, uma maneira que permita rodar o treino e o teste diversas vezes e, a partir disso, obter uma estimativa.

Em vez de um ponto, queremos um intervalo. Precisamos encontrar um método, um algorítimo para isto. Adiante, estudaremos como solucionar essa questão!
Anteriormente, percebemos que, se usarmos a técnica de separar uma parte dos dados para treino e outra para testes, seguramos uma parte dos dados. Em inglês, isto é chamado de Holdout.

Por exemplo, se temos 12 elementos para analisar com o algoritmo, podemos separar os 9 primeiros (de 01 a 09) para treino, e os 3 restantes (de 10 a 12) para testes.

| 01 | 02 | 03 | 04 | 05 | 06 | 07 | 08 | 09 | 10 | 11 | 12 |

Nesse teste, obteríamos o péssimo resultado de 35%. Como falado, isto pode ter sido fruto de uma aleatoriedade, sorte ou azar, porque cada elemento tem uma característica. Então, se tivéssemos separado outros elementos para treino e outros elementos para testes, como 04, 07 e 09 para teste e o restante (01, 02, 03, 05, 06, 08, 10, 11 e 12) para treino:

| 01 | 02 | 03 | 04 | 05 | 06 | 07 | 08 | 09 | 10 | 11 | 12 |

Nesse caso, o resultado seria 78%. Ou ainda, se fizéssemos outra seleção, pegando 02, 04 e 11 para teste, deixando o restante para treino.

| 01 | 02 | 03 | 04 | 05 | 06 | 07 | 08 | 09 | 10 | 11 | 12 |

Então, teríamos 70% como resultado. Que horrível! Por quê? Porque, a cada mudança, ficamos suscetíveis a um número aleatório que define o que será escolhido para treino ou teste, ou de repente à ordem original da sequência — se não fizermos um shuffle — e isto é extremamente arriscado, pois ficamos suscetíveis a esse detalhe horrível, para tomar uma decisão super importante.

Outro exemplo do nosso dia a dia, é quando precisamos fazer uma prova. Imagine que no dia precedente à prova, você receba uma notícia pessoal desagradável, ou acabe brigando com alguém do seu convívio, e, por essa razão, você vá mal na prova e tire 3.5 de 10. Mas se um fator externo afetou seu resultado, não necessariamente significa que você não saiba o conteúdo. Porém, pense: durante um semestre, ou um ano, costuma-se fazer apenas uma prova? Não! É costume que façamos, pelo menos, duas provas e, a partir da soma e divisão do valor de ambas, conseguimos a média!

Então, repare que, no momento em que testamos o algoritmo — ou um(a) aluno(a), por exemplo — diversas vezes, começamos a diminuir a influência de alguns fatores externos. No caso da prova, pode ser alguma situação pessoal; ou no caso de um algoritmo que estivermos rodando, pode ser o shuffle, entre outros problemas. Ou seja, ao rodar o teste várias vezes, podemos tirar uma média simples. Por exemplo, em três provas, considerando que as notas obtidas em cada uma foram: 3.50, 7.80 e 7.00, a média simples delas seria 6.10 (resultado da soma das três notas, dividido por 3).

E podemos fazer a mesma coisa em nosso algoritmo de treino e teste. Em vez de rodar uma única vez o treino e o teste, vamos rodar dois testes, ou seja, treinaremos e testaremos duas vezes e tiraremos a média.

Isso é, pegamos os 12 elementos e definimos que eles serão divididos em dois — podemos dividir entre a primeira metade e a segunda —, treinamos com os 6 primeiros e testamos com os outros 6. O resultado será de 55%. Em ordem contrária, o resultado é de 78%, logo, a média é 66.5% (resultado da soma das duas porcentagens dividido por 2).

Repare que, desta maneira, nós começamos a diminuir a influência de aleatoriedades nas posições, ou talvez de uma pré determinação das posições dos elementos, sobre o resultado. Em vez de apenas separarmos treino e teste e fazermos o holdout, separaremos em N pedaços (aqui, em 2), para treinar, testar e validar o algoritmo de maneira cruzada. O nome deste processo é Cross Validation, e há diversas maneiras de colocá-lo em prática.

No último exemplo, foram separados dois pedaços e feitos dois testes. Poderíamos ter separado em 4 pedaços e, se fosse o caso, teríamos 4 conjuntos de 3. Então treinaríamos com 3/4 e testaríamos com um 1/4. O resultado seria de 35%. Ao treinar com o 1/4 restante, obteríamos o resultado de 88%. Ao separar 1/4 de novo, obteríamos 76%. Continuando com a separação de outra parcela, obteríamos 74%. Considerando estes valores, a média seria de 68.25%. Então, reparem que, em vez de quebrar em dois pedaços e executar duas vezes o treino-teste, agora quebramos em 4 pedaços e executamos o treino-teste 4 vezes também.

Aqui, por praticidade, foram escolhidos 12 elementos para que fossem divisíveis por 2 e por 4, mas não precisava ser exatamente o mesmo número, poderia ser 3, 3, 3 e 2, se tivéssemos só 11 elementos, por exemplo. O principal aqui é a ideia de que quebrarmos os dados em N pedaços e, então, rodarmos K processos de treino e teste, validando nosso algoritmo de maneira cruzada. Esse processo é chamado de K-fold, referente ao número (K) de vezes que quebraremos os dados para rodar a validação cruzada.

Reparem que, se dividirmos em 8 pedaços, e tivermos N elementos e rodarmos todos eles, teremos 8 números diferentes. Por exemplo, se obtivermos 65%, 76%, 74%, 70%, 66%, 62%, 68% e 69%, a média será de 68.75%. Porém, a média é um ponto.

Nos cursos básicos de estatística, como temos na Alura, percebemos como ela é uma medida difícil de utilizarmos secamente, pontualmente. O mais desejável é usar um intervalo no qual confiamos ter um número razoável para o algoritmo funcionar no mundo real.

Então, quando temos um conjunto de números como esse, que contém 8, podemos usar a média e o desvio padrão, se acreditarmos que essa distribuição é normal. Ambos desviam padrões, e nós temos um intervalo entre 60% e 77%, nesse caso específico.

Nós faremos a construção dessa média em Python, logo a seguir.

Por fim, pense em um caso extremo, no qual temos 12 elementos e os quebramos ao máximo, em 12 pedaços. Assim, em um dos 12 treinos e testes, treinaremos com 11 deles e testaremos com 1. Ou seja, vamos testar o número de elementos que tivermos N vezes. Este algoritmo será muito lento, porque esse processo de validar, deixando um isolado, nos obriga a rodá-lo K vezes (o número de elementos que possuímos).

Entretanto, o processo é o mais fiel ao que esperamos com esse algoritmo no mundo real. Sendo assim, é muito importante decidirmos qual será o número K. Esse caso específico, em que o número K é exatamente o número de elementos que temos, é chamado de Leave One Out -"deixar um de fora", em português -, justamente por deixar um elemento de fora, isolado.

In [9]:
from sklearn.model_selection import cross_validate 

In [10]:
from sklearn.model_selection import cross_validate

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3)
results

{'fit_time': array([0.0118854 , 0.00731039, 0.0099225 ]),
 'score_time': array([0.00351739, 0.00228119, 0.00235081]),
 'test_score': array([0.75704859, 0.7629763 , 0.75337534])}

In [11]:
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 10 = [74.24, 77.32]


In [12]:
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [75.21, 76.35]


In [13]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 3 = [%.2f %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) *100))

Accuracy com cross validation, 3 = [75.21 76.35]


Anteriormente, entendemos a importância de rodar mais de um conjunto treino-teste para obter um intervalo, no qual tenhamos um valor aproximado da taxa de acerto. Então, vamos analisar como implementar isso com SKLearn. Joguemos sklearn cross validation em nossa página de pesquisa — no curso, utilizamos o Google.

Queremos saber nossa nota, o score da Cross Validation, então precisamos do cross_validate, que encontramos no segundo resultado da busca. Devemos importar a função sklearn.model_selection.cross_validate. Assim, validamos o modelo para que ele possa ser aplicado e definido.

from sklearn.model_selection import cross_validate COPIAR CÓDIGO
Agora, queremos criar nossa árvore (modelo) igual ao anterior. Para isso, podemos copiar e colar e, em seguida, fazer o cross_validate com o modelo X e Y. Passaremos eles juntos para que a validação cruzada os quebre em pedaços. Vamos indicar que desejamos que ela seja rodada três vezes, ou seja, seu número cv será igual a 3. E, por fim, teremos o resultado, imprimindo os results.

from sklearn.model_selection import cross_validate

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3)
resultsCOPIAR CÓDIGO
Ao executar, obteremos:

{'fit_time': array([0.0075433, 0.00743294, 0.00655079]),
 'score_time': array([0.00128651, 0.0011363, 0.00116062]),
 'test_score': array([0.75504859, 0.7629763, 0.75337534]),
 'train_score': array([0.75832583, 0.75536223, 0.76016199])}COPIAR CÓDIGO
Diversos valores foram impressos, como:

tempo de execução (fit_time);
tempo para nota (score_time);
resultados de teste (test_score);
resultados de treino (train_score).
Se analisarmos a validação cruzada, encontramos o parâmetro return_train_score:

sklearn.model_selection. cross_validate(estimator, X, y=None, groups=None, scoring=None, cv=’warn’, n_jobs=None, verbose=0, fit_params=None, pre_dispatch=‘2*n_jobs’, return_train_score=’warn’, return_estimator=False, error_score=’raise-deprecating’) ¶

Que retorna a nota de treino. Vamos deixá-lo como falso, porque desejamos apenas a nota de teste.

results = cros_validate(modelo, x, y, cv = 3, return_train_score=False)COPIAR CÓDIGO
Ao executar novamente, ele devolve as notas anteriores, com exceção de train_score:

{'fit_time': array([0.01153445, 0.00655961, 0.00689912]),
 'score_time': array([0.00188303, 0.00130415, 0.00126171]),
 'test_score': array([0.75704859, 0.7629763, 0.75337534])}COPIAR CÓDIGO
Percebam que foi devolvido foi um dicionário do Python. Por essa razão, pegaremos teste_score como results.

results['test_score']COPIAR CÓDIGO
Notem que teremos 3 notas como resultado, porque treino e teste foram separados e rodados 3 vezes.

array ([0.75704859, 0.7629763, 0.75337534])COPIAR CÓDIGO
Rodaremos o SEED fixo, assim, os valores se repetem e mantemos a reprodutibilidade de código. Dados esses três valores, queremos saber a média. Para tirarmos a media, basta chamarmos um array do NumPy, por meio de mean().

media = results ['test_score'].mean()COPIAR CÓDIGO
Com isso, obtemos a media de 0.7578000751484867. Todavia, não é apenas a media que queremos, mas o intervalo.

Se voltássemos para o curso de estatística da Alura, no qual fala-se sobre média e desvio padrão, veríamos que 95% da densidade estará dentro da média, mais ou menos dois desvios padrões. Então, desvio padrão é o quanto estamos desviando dessa média. No caso do NumPy, é o mesmo array utilizado acima, mais .std().

A média menos duas vezes o desvio_padrao, e a média mais duas vezes o desvio_padrao vão formar o intervalo que queremos imprimir. Por meio de print(), com objeto Accuracy, imprimiremos dois números com ponto flutuante 2. O primeiro será a média menos duas (- 2) vezes (*) o desvio padrão e o segundo será a media mais duas (+ 2) vezes (*) o desvio padrão.

media = results ['test_score'].mean().
desvio_padrao = results['test_score'].std()
print("Accuracy %.2f %.2f" % (media - 2 * desvio_padrao, media + 2 * desvio_padrao))COPIAR CÓDIGO
Ao executar o código, obteremos um intervalo entre 0.75 e 0.77:

Accuracy 0.75 0.77COPIAR CÓDIGO
Aplicaremos algumas melhorias no código:

colocando %.2f %.2f entre colchetes ([]);
adicionando uma vírgula (,) entre %.2f %.2f;
multiplicando (*) os resultados da subtração e adição da média por 100 para que o Accuracy — intervalo que desejamos utilizar — não seja um número decimal.
Agora, para obtermos o intervalo, executaremos o código todo, da seguinte forma:

from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
media = results ['test_score'].mean().
desvio_padrao = results['test_score'].std()
print("Accuracy [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))COPIAR CÓDIGO
Após aplicar as alterações, se rodarmos o código, obteremos um intervalo entre 74.99 e 76.57:

Accuracy [74.99, 76.57]COPIAR CÓDIGO
Inclusive, podemos rodar novamente, que continuaremos obtendo o mesmo resultado. Maravilha! Esse é o intervalo que queremos utilizar e que, no caso, é igual (=) ao Accuracy com cross validation, 3, que podemos especificar em print():

print("Accuracy com cross validation, 3 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))COPIAR CÓDIGO
Se alterarmos o valor do SEED, de 158020 para 5, continuaremos obtendo como resultado:

Accuracy com cross validation, 3 [74.99, 76.57]COPIAR CÓDIGO
Percebam que mudamos o valor de SEED — número aleatório que influencia o algorítmo —, mas Accuracy se mantém. Podemos alterar novamente SEED, para 301, por exemplo, e o intervalo permanecerá o mesmo, de 74.99 a 76.57. Ou seja, nosso algorítimo não está mais tão suscetível a essa aleatoriedade! Contudo, estamos suscetíveis ao número de cv.

Vamos analisar o que acontece se mudarmos o valor de cv, de 3 para 10.

from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results ['test_score'].mean().
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))COPIAR CÓDIGO
Após alterar o valor de cv, ao executar o código, obteremos:

Accuracy com cross validation, 10 = [74.24, 77.32]COPIAR CÓDIGO
Agora, vamos testar com outro número? Mudemos cv de 10 para 5.

from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results ['test_score'].mean().
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))COPIAR CÓDIGO
Com cv = 5, ao rodar o código, teremos impresso:

Accuracy com cross validation, 5 = [75.21, 76.35]COPIAR CÓDIGO
Então, qual número devemos usar para o cross validation (cv)? Alguns papers científicos, encontrados na documentação da versão usada no vídeo, apontam que escolher 5 ou 10 para cv já é o suficiente. Sendo assim, vamos manter o valor de cv do nosso código como 5, que gerou o intervalo de 75.21 a 76.35, para a taxa de acerto que acreditamos alcançar com o algoritmo que estamos utilizando, como demonstrado logo acima.

A partir da versão 0.22 do SKLearn, passou-se a adotar 5 como padrão, em vez de 3, de acordo com o seguinte trecho da documentação:

Changed in version 0.20: cv default value if None will change from 3-fold to 5-fold in v0.22.

Por essa razão, 5 é o valor que costuma ser utilizado no início de um cross_validate.

Então, resolvemos o problema da suscetibilidade à influencia de um número aleatório — que não faz sentido — e definimos o número 5 para estabelecer quantas vezes rodaremos os processos de treino e teste, que determinará o intervalo, a média e o desvio padrão que passaremos aos nossos clientes.
Ainda que nosso algorítimo esteja funcionando, vamos revisar o código e nos atentar aos seus detalhes:

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))COPIAR CÓDIGO
Reparem que estamos usando um DecisionTreeClassifier() e o cross_validate(). Entretanto, se formos ao scikit learn, encontraremos o seguinte source:

sklearn.model_selection. cross_validate(estimator, X, y=None, groups=None, scoring=None, cv=’warn’, n_jobs=None, verbose=0, fit_params=None, pre_dispatch=‘2*n_jobs’, return_train_score=’warn’, return_estimator=False, error_score=’raise-deprecating’) ¶

Notem que o cross_validate() não recebe o parâmetro de aleatoriedade, e esse é o padrão. Da maneira utilizada por nós, ele é determinístico, e nós sabemos em quantos pedaços ele quebrará os dados, porque somos nós quem decidimos essa quantidade. Portanto, a realidade é que a aleatoriedade (random) só é aplicada em DecisionTreeClassifier(). E, por padrão, ele só a usará em um caso específico e raro. Por estas razões, mesmo que mudemos constantemente o valor de SEED, obteremos basicamente sempre os mesmos números.

A ideia era rodar aleatoriamente, de fato. Por mais que recebamos todos os dados em uma sequência, não queremos quebrar eles em 5 pedaços e, a partir disso, fazer o cross_validate. O ideal seria embaralhar estes dados e, então executar a validação cruzada. O algorítimo pode fazer isso de maneira mais inteligente, sem embaralhar e copiar todo o espaço de memória e todos os objetos que estiverem dentro, e assim por diante. Mas o importante é a ideia de embaralhar esses dados de alguma maneira, e o cross_validate não faz isso de verdade.

Para começarmos a resolver esta questão, adicionemos uma célula de texto, clicando em "TEXT > CELL (com seta para cima)", abaixo do algorítimo. Em seguida, adicionaremos a cerquilha (#) seguida por Aleatoriedade no cross validate, intitulando nossa ação:

# Aleatoriedade no cross validateCOPIAR CÓDIGO
Depois, copiaremos o código anterior e o colaremos abaixo. Por fim, modificaremos o cv de 5 para 10 — valor que será mantido de agora em diante.

#Aleatoriedade no cross validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results ['test_score'].mean().
desvio_padrao = results['test_score'].std()
print(Accuracy com cross validation, 10 = [%.2f, %.2f] % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))COPIAR CÓDIGO
O que desejamos modificar é a aleatoriedade, nós desejamos que ele embaralhe nossos dados, entretanto, não existe um parâmetro para isto. Todavia, o cv aceita como parâmetro, tanto números, quanto geradores de validação cruzada. E existem diversos geradores de validação cruzada. O que estamos usando é o KFold, que "corta" nossos dados em K pedaços. Por sorte, o sklearn tem o KFold, no modo selection:

class sklearn.model_selection.KFold(nsplits=’warn’, shuffle=False, randomstate=None)

É ele que estamos usando por baixo dos panos, ou seja, sorrateiramente. Então, por enquanto, nosso cross_validate é um KFold de n_splits, no caso, igual a 10. Vamos rodar o código, que com essas modificações, ficou da seguinte forma:

from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()* desvio_padrao) * 100))
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))COPIAR CÓDIGO
Como retorno, obteremos:

Accuracy com cross validation, 10 = [74.37, 77.19]COPIAR CÓDIGO
Agora, o ideal é pegarmos os valores da média de desvio padrão e colocarmos em uma função, para facilitarmos nosso trabalho. Nossa função será definida como:

def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()* desvio_padrao) * 100))
    print("Accuracy com cross validation = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))COPIAR CÓDIGO
Assim, ela recebe os results inteiros e, em seguida faz a média, o desvio padrão e imprime os resultado. Neste caso, podemos copiar e colar esta parte do código.

Para não dependermos de uma variável extra, excluímos 10 de print(). E, no código acima, adicionaremos uma linha ao fim do código, para que o algorítimo chame-se apenas imprime_resultados() de modo que não tenhamos de fazer manualmente sempre que formos rodar o código. Ele diretamente pegará a média, o desvio padrão e os imprimirá.

from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)COPIAR CÓDIGO
Podemos subir o código de imprime_resultados() para cima, selecionando-o e pressionando o botão "CELL (com a seta para cima)", para que a definição venha em primeiro. A ordenação ficará da seguinte forma:

def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy com cross validation = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100))COPIAR CÓDIGO
E, logo abaixo:

from sklearn.model_selection import KFold
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)COPIAR CÓDIGO
E rodemos o código logo acima novamente, da mesma forma que estávamos fazendo, com KFold de n_splits igual a 10. Como retorno, obteremos:

Accuracy com cross validation = [74.37, 77.19]COPIAR CÓDIGO
Repare como ficou difícil de acharmos a média. Por esta razão, vamos adicionar um print() para ela em imprime_resultados(). Para especificar, mudaremos os nomes dos prints para Accuracy médio e Accuracy intervalo, respectivamente.

def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio: %.2f" % (media * 100).)
    print("Accuracy intervalo: [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100))COPIAR CÓDIGO
Agora, ao rodar, teremos:

Accuracy médio: 75.78
Accuracy intervalo: = [74.37, 77.19]COPIAR CÓDIGO
Legal! Está claro que a média é 75.78, que está entre o intervalo de 74.37 a 77.19.

Ainda não conseguimos embaralhar nossos dados, porém, com o objeto KFold criado, será possível fazermos um suffle, pois ele tem um randon_state para ser setado.

class sklearn.model_selection.KFold(nsplits=’warn’, shuffle=False, randomstate=None) ¶

Vamos copiar e colar o código e adicionar o shuffle como parâmetro de KFold(), juntamente com n_splits para vermos quais resultado atingiremos com e sem o shuffle. Primeiro, executaremos com shuffle, ou seja, shuffle = True:

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y cv = cv, return_train_score=False)
imprime_resultados(results)COPIAR CÓDIGO
Ao executar o código com shuffle, obteremos outros resultados:

Accuracy médio: 75.76
Accuracy intervalo: [73.26, 78.26]COPIAR CÓDIGO
É importante ressaltar que se rodarmos diversas vezes, Accuracy se mantém, pois continua sendo determinístico. Mas ainda assim, é diferente dos valores apresentados sem o shuffle. No entanto, repare que a média obtida por meio dos dois algorítimos é bastante próxima, mas o intervalo é diferente. Na prática, se os dados não possuem alguma estrutura interna, como uma sequência por datas, por exemplo, primeiro gera-se um sequência aleatória desses dados e, em seguida, quebra-se em 10 pedaços ou faz-se ambos ao mesmo tempo. Esta é a forma tradicional de execução com shuffle e n_splits.

Assim, podemos usar não somente o KFold, mas, em seu Model Selection, encontraremos o KFold e diversos outros "quebradores" de grupos, que são as Splitter classes. Estudaremos alguns destes outros adiante.

In [14]:
#Cross validate
#Aleatoriedade no cross validate
from sklearn.model_selection import KFold


SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print('Accuracy com cross validation, 10 = [%.2f, %.2f]' % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 10 = [74.24, 77.32]


In [15]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio: %.2f" % (media * 100))
    print("Accuracy intervalo: [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100))

In [16]:

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [75.21, 76.35]


In [17]:
SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 10 = [74.24, 77.32]


In [18]:
#Cross validate
#Aleatoriedade no cross validate
from sklearn.model_selection import KFold

def imprime_resultados(results):
  media = results['test_score'].mean()
  desvio_padrao = results['test_score'].std()
  print("Accuracy médio: %.2f" % (media * 100))
  print("Accuracy intervalo: [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

In [19]:
#Cross validate
#Aleatoriedade no cross validate
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [74.37, 77.19]


In [20]:
#Utilizar o método KFold para embaralhar(shuffle).

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.76
Accuracy intervalo: [73.26, 78.26]


In [21]:
#Simular situação horrível de azar

#Pode ser uma situação de "azar" ou uma proporção de exemplos desbalanceado entre as classes.
# Ordenar nosso dataframe utilizando o sort_value(“vendido“’);


dados.sort_values("vendido", ascending=True)

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704
...,...,...,...,...
5491,71910.43,1,9,25778.40812
1873,30456.53,1,6,15468.97608
1874,69342.41,1,11,16909.33538
5499,70520.39,1,16,19622.68262


In [22]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [23]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 57.84
Accuracy intervalo: [34.29, 81.39]


In [24]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [72.30, 79.26]


In [25]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.55, 78.01]


In [26]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.83, 77.73]


Sempre podemos ter muito azar em uma condição muito específica. Antes de trabalharmos com validação cruzada, estudamos uma questão de sorte e azar em que a validação era simples: treino-teste. Tínhamos o azar de que nossos dados, no momento de realmente utilizá-los, eram separados como, por exemplo, vendido ou não vendido.

Como resultado, no treino estavam quase todos que não foram vendidos e no teste quase todos que foram vendidos. Ou seja, ocorria um desbalanceamento das classes, seja no conjunto inteiro de dados ou na hora de treinar e testar, e seja pela aleatoriedade ou pela natureza dos dados. Então, se tivermos azar, o que pode acontecer?

Quando separávamos o treino e o teste com o train_test_split — que também está no model_selection, em Splitter Functions — tínhamos o Stratify como parâmetro, que definia a proporção. Por exemplo, se temos 70% dos dados na classe 0 (não vendido) e 30% na classe 1 (vendido), ele distribui 70% e 30% tanto para treino, quanto para teste. No entanto, esse parâmetro não existe no cross_validate do KFold, que possui apenas os parâmetros n_splits, shuffle e random_state. Sendo assim, não está estratificando.

Então, vamos simular uma situação de azar para podermos aprimorar nosso processo. Sendo assim, identificaremos esse código com # Simular situação horrível de azar. Abaixo, adicionaremos o texto Pode ser uma situação de "azar" ou uma proporção de exemplos desbalanceado entre as classes. Pegaremos os dados (dados.sort_values()) do nosso data frame e ordenaremos pela classe vendido, crescente (ascending=True), ou seja, primeiro 0 e depois 1. Assim, o "não vendido" virá antes do "vendido". Essa é a separação que estabelecemos.

#Simular situação horrível de azar

#Pode ser uma situação de "azar" ou uma proporção de exemplos desbalanceado entre as classes.

dados.sort_values("vendido", ascending=True)COPIAR CÓDIGO
Ao executar o código acima, obteremos a seguinte tabela:

preco	vendido	idade_do_modelo	km_por_ano
4999	74023.29	0	12	24812.80412
5322	84843.49	0	13	23095.63834
5319	83100.27	0	19	36240.72746
5316	87932.13	0	16	32249.56426
5315	7937.01	0	15	28414.50704
5310	62474.09	0	19	26142.11896
...	...	...	...	...
5459	42806.87	1	20	22194.40794
5458	83117.55	1	15	18068.06018
5485	58506.40	1	16	8946.32106
5486	37167.26	1	14	12393.52734
1875	50891.00	1	12	12517.44852
...	...	...	...	...
Conseguimos ordenar de maneira que, primeiro, apareçam todos os veículos não vendidos (0) e posteriormente todos os vendidos (1), ambos em ordem crescente. Esses dados serão chamados de dados_azar. O x_azar será como o x anterior, composto por preco, idade_do_modelo e km_por_ano, enquanto y_azar será vendido. E pegaremos só os primeiros dados_azar, por meio de head().

dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()COPIAR CÓDIGO
Ao rodar esse código, obteremos os cinco primeiros elementos, todos classificados com 0:

preco	vendido	idade_do_modelo	km_por_ano
4999	74023.29	0	12	24812.80412
5322	84843.49	0	13	23095.63834
5319	83100.27	0	19	36240.72746
5316	87932.13	0	16	32249.56426
5315	77937.01	0	15	28414.50704
Agora, vamos rodar o KFold sem o shuffle, e com esses dados de azar.

from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)COPIAR CÓDIGO
Ao executar o código acima, sem o shuffle, obteremos:

Accuracy médio: 57.84
Accuracy intervalo: [34.29, 81.39]COPIAR CÓDIGO
A acurácia é baixíssima! Vamos tentar com o shuffle agora:

from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)COPIAR CÓDIGO
Com shuffle=True obteremos:

Accuracy médio 75.78
Accuracy intervalo: [72.30 79.26]COPIAR CÓDIGO
O shuffle nos é suficientemente bom nesta situação em que temos balanço para recuperar, mas e se quiséssemos fazer a estratificação?

Além do KFold, temos outros geradores de grupo de separação, dentre eles o StratifiedKFold, que recebe um número de n_splits, shuffle e depois separa, mantendo a proporção entre as classes.

class sklearn.model_selection.StratifiedKFold(nsplits=’warn’, shuffle=False, randomstate=None)

É este que usaremos! Então, copiaremos e colaremos nosso algorítimo e, em cv, modificaremos KFold para StratifiedKFold, lembrando de importá-lo no início do código.

from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)COPIAR CÓDIGO
Rodaremos esse código e obteremos:

Accuracy médio 75.78
Accuracy intervalo: [72.94 78.62]COPIAR CÓDIGO
Embora o resultado seja mais fechado, é o mais recomendado. Principalmente quando há um desbalanço entre duas ou mais classes, é interessante utilizar o StratifiedKFold.

Reparem como ficariam os resultados se o deixassemos no aleatório. Inclusive, mesmo sem o shuffle, ele recupera bem a capacidade de prever o quão bom seria o treinamento. Vamos deletar shuffle:

from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)COPIAR CÓDIGO
E executar o código. Nesse caso, teremos o seguinte retorno:

Accuracy médio 75.78
Accuracy intervalo: [73.83 77.73]COPIAR CÓDIGO
Feita essa demonstração, podemos voltar o shuffle ao código, mas será o StrafiedKFold que mais usaremos em casos de desbalanço entre as classes.

In [27]:
np.random.seed(SEED)
np.random.randint(-2, 3)

-2

In [28]:
np.random.seed(SEED)
np.random.randint(-2, 3, size=10000)

array([-2,  2,  0, ...,  0,  2,  2])

In [29]:
np.random.seed(SEED)
dados["modelo_aleatorio"] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


In [30]:
dados.modelo_aleatorio.unique()

array([16, 22, 12,  4,  3, 11, 18, 17, 13,  0, 15, 10,  9, 14,  1,  5, 19,
       21,  8,  7, 20,  6,  2, -1])

In [31]:
dados.modelo_aleatorio.min()

-1

In [32]:
abs(dados.modelo_aleatorio.min())

1

In [33]:
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min())
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,17
1,40557.96,1,20,12622.05362,23
2,89627.50,0,12,11440.79806,13
3,95276.14,0,3,43167.32682,5
4,117384.68,1,4,12770.11290,4


In [34]:
dados.modelo_aleatorio.min()

0

In [35]:
np.random.seed(SEED)
dados['modelo_aleatorio'] = dados.idade_do_modelo +np.random.randint(-2, 3, size=10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


In [36]:
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [37]:
dados.modelo_aleatorio.min()

1

In [38]:
abs(dados.modelo_aleatorio.min())

1

In [39]:
np.random.seed(SEED)
dados['modelo_aleatorio'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [40]:
dados.modelo_aleatorio.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

Gerar dados aleatórios com o randint;
Gerar uma nova coluna de dados;
Utilizar o comando unique para mostrar os modelos únicos;
Utilizar o GroupKFold para agrupar por grupo.